In [ ]:
# Drive version:
# !pip uninstall TabularExperimentTrackerClient --y
# !pip install git+https://github.com/DanielWarfield1/TabularExperimentTrackerClient
# !pip uninstall NeuralNetworksTrainingPackage --y
# !pip install git+https://github.com/Bartosz-G/NeuralNetworksTrainingPackage

In [1]:
%pip uninstall NeuralNetworksTrainingPackage --y
#%pip uninstall TabularExperimentTrackerClient --y
%pip install git+https://github.com/DanielWarfield1/TabularExperimentTrackerClient
%pip install git+https://github.com/Bartosz-G/NeuralNetworksTrainingPackage

Found existing installation: NeuralNetworksTrainingPackage 1.0.0
Uninstalling NeuralNetworksTrainingPackage-1.0.0:
  Successfully uninstalled NeuralNetworksTrainingPackage-1.0.0
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/DanielWarfield1/TabularExperimentTrackerClient to /tmp/pip-req-build-sl0_m1ar
  Running command git clone --filter=blob:none --quiet https://github.com/DanielWarfield1/TabularExperimentTrackerClient /tmp/pip-req-build-sl0_m1ar
  Resolved https://github.com/DanielWarfield1/TabularExperimentTrackerClient to commit 780933411aa8c4e394478a26dec4a39447f8f012
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/Bartosz-G/NeuralNetworksTrainingPackage to /tmp/pip-req-build-eupjlw2s
  Running command git clone --filter=blob:none --qui

In [4]:
import numpy as np
import pandas as pd
import sklearn
import torch
import time

In [5]:
import os
from TabularExperimentTrackerClient.ExperimentClient import ExperimentClient

path =  '../creds/'
creds_orch_file = "creds-orch.txt"
creds_openml_file = "creds-openml.txt"



with open(os.path.join(path, creds_orch_file), 'r') as file:
    lines = file.readlines()
    orchname = lines[0].strip()
    orchsecret = lines[1].strip()

with open (os.path.join(path, creds_openml_file), "r") as myfile:
    openMLAPIKey = myfile.read()

ex = ExperimentClient(verbose = True)


ex.define_orch_cred(orchname, orchsecret)
ex.define_opml_cred(openMLAPIKey)

# Colab version
# ex.define_opml_cred_drive('/My Drive/research/non-homogenous-data/creds/creds-openml.txt')
# ex.define_orch_cred_drive('bart', '/My Drive//research/non-homogenous-data/creds/creds-colab.txt')

# 1. Defining the experiment

In [11]:
experiment_name = 'experiment_LCN_1'



# LCN and LLN Parameters
depth = {'distribution': 'int_uniform', 'min':1, 'max':11}
seed = {'distribution': 'constant', 'value': 42}
drop_type = {'distribution': 'categorical', 'values':['node_dropconnect', 'none']}
p = {'distribution': 'float_uniform', 'min':0.25, 'max':0.75}
back_n = {'distribution': 'categorical', 'values':[0, 0, 0, 1]}
hidden_dim = {'distribution': 'constant', 'value': 1} # Assertion error coming from Net if not 1
anneal = {'distribution': 'categorical', 'values':['interpolation', 'none', 'approx']}
batch_size = {'distribution': 'categorical', 'values':[16,32,64,64,64,128,256]}
epochs = {'distribution': 'constant', 'value': 30}
lr = {'distribution': 'log_uniform', 'min':0.05, 'max':0.2} # yields mean = 0.1082, median 0.1
momentum = {'distribution': 'constant', 'value': 0.9}
no_cuda = {'distribution': 'constant', 'value': False}
lr_step_size = {'distribution': 'categorical', 'values':[10, 10, 15, 20]}
gamma = {'distribution': 'constant', 'value': 0.1}

In [12]:
#============================================================
# Regression: Locally Constant Networks
#============================================================
LCN_reg_SGD_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'SGD'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'regression'}
    }

LCN_reg_AMSGrad_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'AMSGrad'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'regression'}
    }

#============================================================
# Regression: Locally Linear Networks
#============================================================

LLN_reg_SGD_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_linear'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'SGD'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'regression'}
    }


LLN_reg_AMSGrad_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_linear'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'AMSGrad'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'regression'}
    }

#============================================================
# Classification: Locally Constant Networks
#============================================================

LCN_cls_SGD_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'SGD'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'classification'}
    }

LCN_cls_AMSGrad_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_constant'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'AMSGrad'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'classification'}
    }

#============================================================
# Classification: Locally Linear Networks
#============================================================

LLN_cls_SGD_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_linear'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'SGD'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'classification'}
    }


LLN_cls_AMSGrad_space = {
    'depth': depth,
    'seed': seed,
    'drop_type': drop_type,
    'p': p,
    'ensemble_n': {'distribution': 'constant', 'value': 1},
    'shrinkage': {'distribution': 'constant', 'value': 1},
    'back_n': back_n,
    'net_type': {'distribution': 'constant', 'value': 'locally_linear'},
    'hidden_dim': hidden_dim,
    'anneal': anneal,
    'optimizer': {'distribution': 'constant', 'value': 'AMSGrad'},
    'batch_size': batch_size,
    'epochs': epochs,
    'lr': lr,
    'momentum': momentum,
    'no_cuda': no_cuda,
    'lr_step_size': lr_step_size,
    'gamma': gamma,
    'task': {'distribution': 'constant', 'value': 'classification'}
    }

In [13]:
model_groups = {
    'LCN_reg_SGD':{'model':'LCN_reg_SGD', 'hype':LCN_reg_SGD_space},
    'LCN_reg_AMSGrad':{'model':'LCN_reg_AMSGrad', 'hype':LCN_reg_AMSGrad_space},
    'LLN_reg_SGD':{'model':'LLN_reg_SGD', 'hype':LLN_reg_SGD_space},
    'LLN_reg_AMSGrad':{'model':'LLN_reg_AMSGrad', 'hype':LLN_reg_AMSGrad_space},
    'LCN_cls_SGD':{'model':'LCN_cls_SGD', 'hype':LCN_cls_SGD_space},
    'LCN_cls_AMSGrad':{'model':'LCN_cls_AMSGrad', 'hype':LCN_cls_AMSGrad_space},
    'LLN_cls_SGD':{'model':'LLN_cls_SGD', 'hype':LLN_cls_SGD_space},
    'LLN_cls_AMSGrad':{'model':'LLN_cls_AMSGrad', 'hype':LLN_cls_AMSGrad_space},
}

ex.def_model_groups(model_groups)

In [14]:
ex.def_data_groups_opml()
print(f'automatically defined data groups: {ex.data_groups.keys()}')

classification_models = [k for k in model_groups.keys() if '_cls' in k]
regression_models = [k for k in model_groups.keys() if '_reg' in k]


applications = {'opml_reg_purnum_group': regression_models,
                'opml_reg_numcat_group': regression_models,
                'opml_class_purnum_group': classification_models,
                'opml_class_numcat_group': classification_models}

ex.def_applications(applications)
ex.reg_experiment(experiment_name)

automatically defined data groups: dict_keys(['opml_reg_purnum_group', 'opml_class_purnum_group', 'opml_reg_numcat_group', 'opml_class_numcat_group'])
existing experiment found


'existing experiment found'

In [15]:
exp_info = ex.experiment_info()
successful_runs = exp_info['successful_runs']
required_runs = exp_info['required_runs']
print('total required runs: {}'.format(required_runs))

total required runs: 14160


In [15]:
# data_pre_processing.reset(True)

# 2. General Data pre-processing

In [16]:
from NeuralNetworksTrainingPackage.event_handler import dataPreProcessingEventEmitter
from NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import filterCardinality, quantileTransform, trunctuateData, oneHotEncodePredictors, oneHotEncodeTargets, toDataFrame, splitTrainValTest

n_sample = 20000
split = [0.5, 0.25, 0.25]
quantile_transform_distribution='uniform'


data_pre_processing = dataPreProcessingEventEmitter()

filter_cardinality = filterCardinality(transform = 'all')
trunctuate_data = trunctuateData(n = n_sample, transform = 'all')
one_hot_encode_predictors = oneHotEncodePredictors(transform = 'all')
one_hot_encode_targets = oneHotEncodeTargets(transform = 'all')
to_data_frame = toDataFrame(transform = 'all')
split_train_val_test = splitTrainValTest(split = split)
quantile_transform = quantileTransform(output_distribution = quantile_transform_distribution, transform = 'train')


# Transformations will be called in the order they're added to data_pre_processing
data_pre_processing.add_pre_processing_step('regression', filter_cardinality)
data_pre_processing.add_pre_processing_step('regression', trunctuate_data)
data_pre_processing.add_pre_processing_step('regression', one_hot_encode_predictors)
data_pre_processing.add_pre_processing_step('regression', to_data_frame)
data_pre_processing.add_pre_processing_step('regression', split_train_val_test)
data_pre_processing.add_pre_processing_step('regression', quantile_transform)


data_pre_processing.add_pre_processing_step('classification', filter_cardinality)
data_pre_processing.add_pre_processing_step('classification', trunctuate_data)
data_pre_processing.add_pre_processing_step('classification', one_hot_encode_predictors)
data_pre_processing.add_pre_processing_step('classification', one_hot_encode_targets)
data_pre_processing.add_pre_processing_step('classification', to_data_frame)
data_pre_processing.add_pre_processing_step('classification', split_train_val_test)
data_pre_processing.add_pre_processing_step('classification', quantile_transform)

# 3. Model Specific Data pre-processing

In [17]:
from NeuralNetworksTrainingPackage.dataprocessing.basic_pre_processing import CustomDataset, toPyTorchDatasets

to_pytorch_datasets = toPyTorchDatasets(wrapper = CustomDataset)

# Transformations will be called after general pre-processing steps, and in order they're added
data_pre_processing.add_pre_processing_step('LCN_reg_SGD', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LCN_reg_AMSGrad', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LLN_reg_SGD', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LLN_reg_AMSGrad', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LCN_cls_SGD', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LCN_cls_AMSGrad', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LLN_cls_SGD', to_pytorch_datasets)

data_pre_processing.add_pre_processing_step('LLN_cls_AMSGrad', to_pytorch_datasets)

# 4. Model Metrics

In [18]:
from metrics.LcnMetrics import LcnMetricsClassification, LcnMetricsRegression

lcn_metrics_regression = LcnMetricsRegression()
lcn_metrics_classification = LcnMetricsClassification()

lcn_metrics = {'regression': lcn_metrics_regression,
               'classification': lcn_metrics_classification}

metric_model_pairs = {
    'LCN_reg_SGD': lcn_metrics,
    'LCN_reg_AMSGrad': lcn_metrics,
    'LLN_reg_SGD': lcn_metrics,
    'LLN_reg_AMSGrad': lcn_metrics,
    'LCN_cls_SGD': lcn_metrics,
    'LCN_cls_AMSGrad': lcn_metrics,
    'LLN_cls_SGD': lcn_metrics,
    'LLN_cls_AMSGrad': lcn_metrics,
}

# 5. Model Training routine

In [19]:
from models.LcnNetwork import init_LcnNetwork
from training.LcnTrain import LcnTrainingRoutine

lcn_model_and_training = {'model_init':init_LcnNetwork, 'training_routine': LcnTrainingRoutine}

model_training_pairs = {
    'LCN_reg_SGD': lcn_model_and_training,
    'LCN_reg_AMSGrad': lcn_model_and_training,
    'LLN_reg_SGD': lcn_model_and_training,
    'LLN_reg_AMSGrad': lcn_model_and_training,
    'LCN_cls_SGD': lcn_model_and_training,
    'LCN_cls_AMSGrad': lcn_model_and_training,
    'LLN_cls_SGD': lcn_model_and_training,
    'LLN_cls_AMSGrad': lcn_model_and_training,
}

# 6. Main Experiment loop

In [ ]:


torch_models = ('LCN_reg_SGD', 'LCN_reg_AMSGrad', 'LLN_reg_SGD', 'LLN_reg_AMSGrad', 'LCN_cls_SGD','LCN_cls_AMSGrad', 'LLN_cls_SGD','LLN_cls_AMSGrad')

sklearn_models = ('no_sklearn_models_in_this_training')

for i in range(14160):
    print(f'==== Begin run:{i} ====')
    run_info = ex.begin_run_sticky()

    hyperparameters = run_info['hyp']
    model = run_info['model']

    task = hyperparameters['task']
    seed = hyperparameters['seed']



    torch.manual_seed(seed)
    np.random.seed(seed)


    print('---- Loading datasets ----')
    X, y, categorical_indicator, attribute_names = ex.opml_load_task(run_info['mtpair_task'])

    # Pre-processing
    data_pre_processing.set_seed_for_all(seed)
    data_pre_processing.set_dataset(X, y, categorical_indicator, attribute_names)
    data_pre_processing.apply(task)
    data_pre_processing.apply(model)
    train_data, val_data, test_data = data_pre_processing.get_train_val_test()


    # Getting appropriate metrics
    metrics_calculator = metric_model_pairs[model][task]


    match model:
        case _ if model in sklearn_models:
            pass

        case _ if model in torch_models:
            # hyperparameters will be updated with {'input_dim': num_columns_X, 'output_dim':num_columns_Y}
            hyperparameters.update(train_data.get_dims())

            train_batch_size = hyperparameters['batch_size']
            train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=train_batch_size,shuffle= True)
            val_dataloader = torch.utils.data.DataLoader(val_data,batch_size=len(val_data),shuffle= True)
            test_dataloader = torch.utils.data.DataLoader(test_data,batch_size=len(test_data),shuffle= True)


            init_model = model_training_pairs[model]['model_init']
            TrainingRoutine = model_training_pairs[model]['training_routine']


            model = init_model(**hyperparameters)
            training_routine = TrainingRoutine(**hyperparameters)

            start_epoch = 1  # start from epoch 1 or last checkpoint epoch
            total_epochs = hyperparameters['epochs']
            start_time = time.time()

            for epoch in range(start_epoch, total_epochs + start_epoch):
                print(f"----{epoch}th training epoch ----")
                epoch_metrics = {}

                training_routine.scheduler_step(epoch)
                train_loss = training_routine.train(model, train_dataloader)


                if train_loss is None:
                    print('---Stopping training due to loss being nan!---')
                    epoch_metrics = {'train_loss': train_loss, 'epoch': epoch}
                    ex.update_run(epoch_metrics)
                    break

                if epoch == args.epochs:
                    continue

                epoch_metrics.update(train_loss)
                epoch_metrics.update({'epoch': epoch})
                ex.update_run(epoch_metrics)

            else:
                final_metrics = {}
                training_time = time.time()-start_time

                train_metrics = metrics_calculator.get_metrics(model, device, train_dataloader, 'train')
                val_all_metrics = metrics_calculator.get_all(model, device, val_dataloader, 'val')
                test_all_metrics = metrics_calculator.get_all(model, device, test_dataloader, 'test')

                final_metrics.update(train_loss)
                final_metrics.update(train_metrics)
                final_metrics.update(val_all_metrics)
                final_metrics.update(test_all_metrics)
                final_metrics.update({'epoch': epoch})
                final_metrics.update({'epoch_time': training_time})

                ex.update_run(final_metrics)

    ex.end_run()

## Checking whether the code works as intended

In [20]:
X, y, categorical_indicator, attribute_names = ex.opml_load_task('336-361073')

downloading task 336-361073
task different than previous task, downloading...


In [21]:
from copy import deepcopy
#X_c, y_c, categorical_indicator_c, attribute_names_c = deepcopy(X), deepcopy(y), deepcopy(categorical_indicator), deepcopy(attribute_names)
X, y, categorical_indicator, attribute_names = deepcopy(X_c), deepcopy(y_c), deepcopy(categorical_indicator_c), deepcopy(attribute_names_c)

In [22]:
X_c

,f5,f6,f7,f8,f9,f13,f14,f15,f16,f17,...,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33
0,60,108,76,71,94,0,0,3,0,1,...,0,0,0,0,0,0,0,0,0,0
1,130,77,76,71,94,0,0,0,43,81,...,0,0,0,0,0,0,0,0,0,0
2,110,89,76,71,94,0,6,3,9,48,...,0,0,0,0,0,0,0,0,0,0
3,13,126,89,72,94,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
4,15,119,78,71,94,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,15,116,144,79,94,2,0,0,0,0,...,0,18,23,0,0,0,0,0,0,0
14996,16,137,145,71,94,0,1,2,0,0,...,36,0,0,0,0,0,0,0,0,0
14997,123,77,76,71,94,17,38,0,25,37,...,0,0,0,0,0,0,0,0,0,0
14998,18,118,78,71,94,3,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [23]:
y_c

0        0
1        0
2        0
3        0
4        0
        ..
14995    0
14996    0
14997    0
14998    0
14999    0
Name: foo, Length: 15000, dtype: uint8

In [32]:
categorical_indicator

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [24]:
data_pre_processing.set_seed_for_all(seed)
data_pre_processing.set_dataset(X, y, categorical_indicator, attribute_names)
data_pre_processing.apply(task)

NameError: name 'task' is not defined

In [46]:
print(data_pre_processing.test)

None


In [47]:
X_out, y_out, categorical_indicator_out, attribute_names_out = data_pre_processing.get('train')

In [48]:
X_out

,f5,f6,f7,f8,f9,f13,f14,f15,f16,f17,...,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33
0,60,108,76,71,94,0,0,3,0,1,...,0,0,0,0,0,0,0,0,0,0
1,130,77,76,71,94,0,0,0,43,81,...,0,0,0,0,0,0,0,0,0,0
2,110,89,76,71,94,0,6,3,9,48,...,0,0,0,0,0,0,0,0,0,0
3,13,126,89,72,94,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
4,15,119,78,71,94,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,15,116,144,79,94,2,0,0,0,0,...,0,18,23,0,0,0,0,0,0,0
14996,16,137,145,71,94,0,1,2,0,0,...,36,0,0,0,0,0,0,0,0,0
14997,123,77,76,71,94,17,38,0,25,37,...,0,0,0,0,0,0,0,0,0,0
14998,18,118,78,71,94,3,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [49]:
y_out

0        0
1        0
2        0
3        0
4        0
        ..
14995    0
14996    0
14997    0
14998    0
14999    0
Name: foo, Length: 15000, dtype: uint8